In [ ]:
import pyelabtools_prep as elab
import webbrowser
import os
import elabapi_python
import re
from datetime import datetime

In [ ]:
# Ignore predefined text
IGNORE_MOTIVATION = '<p>Please summarize the aim of the experiment (1-2 lines).</p>'
IGNORE_PREPARATION = '<p>Please answer the preparation questions in the experiment instructions here. Bullet points, sketches and/or formulas are sufficient. Images can be inserted via copy/paste. Formulas can be entered using LaTeX code, e.g.\xa0</p>\n<pre class="language-latex"><code>$$R=\\frac{U}{I}$$</code></pre>\n<p>results in: $$R=\\frac{U}{I}$$</p>'
IGNORE_DESCRIPTION = '<p>Describe the experiment here (e.g. circuit diagrams, experimental protocols, expected results). You may add further details to this section during the experiment (e.g. foto of the setup).</p>'

# Preparation text until the following heading
PREP_UNTIL_HEADING = '<h1>Results</h1>'

# Prefix for group names
GROUPNAME_PREFIX = 'DemoLab SoSe 2024 G'

# Experiment category
CATEGORY_EXPERIMENT = 'Experiment'

In [ ]:
# Which lab course day should be considered for the preparation excerpts?
tag_labcourseday = 'LabCourseDay_1'

# Which groups should be considered? Example: ['001', '002'] or None for all groups
process_groups = None

In [ ]:
# Server and API data for eLabFTW
API_HOST_URL = 'https://your_elabftw_url/api/v2'
# an API key with read permissions is required that was generated in the eLabFTW team of the lab course
API_KEY = 'your_api_key' 

In [ ]:
# Connect to eLabFTW
elab.connect(API_HOST_URL, API_KEY)

# Read and display the team that is related to the API key; this team is used for setting up the lab course structure
team = elab.get_currentteam()
print(f"Team related to the API key: {team.name} (elabftw_teamid = {team.id})")

In [ ]:
# some helper functions

def ensure_dir_exists(dir_name: str):  
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
   
def fixprep(prep: str, ignore=[], shift_heading: int=0):
    ignore.append('<p> </p>')
    for ign in ignore:
        if ign in prep:
            prep = prep.replace(ign, '')
    for headingidx in [5, 4, 3, 2, 1]:
        newheadingidx = headingidx + shift_heading
        if newheadingidx > 6:
            newheadingidx = 6
        if newheadingidx < 1:
            newheadingidx = 1
        prep = prep.replace(f"<h{headingidx}>", f"<h{newheadingidx}>")
        prep = prep.replace(f"</h{headingidx}>", f"</h{newheadingidx}>")
    return prep

def fiximages(html_string: str, expid: int, path: str):
    image_source_string = '<img src="'
    ensure_dir_exists(path+'/images')
    if image_source_string in html_string:
        pos_src_start = html_string.find(image_source_string) + len(image_source_string)
        pos_src_end = pos_src_start + html_string[pos_src_start:].find('"')
        true_src_string = html_string[pos_src_start:pos_src_end]
        true_src_string = true_src_string.replace('%2F','/')
        save_name = 'images/' + true_src_string.split('/')[-1]
        save_name = save_name.split('&')[0]
        long_name = true_src_string.split('f=')[-1]   
        long_name = long_name.split('&')[0]
        elab.save_file_longname(long_name, path+'/'+save_name, expid)
        html_string = re.sub('width="+[1-9]+" ', '', html_string)
        html_string = re.sub('height="+[1-9]+" ', '', html_string)
        html_string = html_string[:pos_src_start] + save_name  + '" style=" max-width: 100%; max-height: 500px' + fiximages(html_string[pos_src_end:], expid, path)
    return html_string

def append_other_uploads(html_string: str, expid: int, path: str):
    uploads = elab.get_uploads(expid)

    html_add = ''
    for upload in uploads:
        longname = upload.long_name
        savename = longname.split('/')[-1]
        if savename in html_string:
            continue
        else:
            elab.save_file_longname(longname, path+'/images/'+savename, expid)
            html_add = html_add + '<p><img src="images/' + savename +'" style=" max-width: 100%; max-height: 500px"'+'></p>\n'
    if html_add != '':
        html_string = html_string + '<h3>Additional images attached</h3>\n' + html_add
    return html_string


In [ ]:
# Generate excerpts

if process_groups is None:
    groups = []
    eln_groups = elab.get_groups()
    for group in eln_groups:
        if group.name.startswith(GROUPNAME_PREFIX):
            groups.append(group.name)
else:
    groups = process_groups

print('Process groups:')
for group in groups:
    print(f"- {group}")
print()

ensure_dir_exists(tag_labcourseday)

for group in groups:
    filename = 'Preparation_' + tag_labcourseday + '_' + group + '.html'
    tag_group = group.replace(' ', '_')

    path = tag_labcourseday + '/' + tag_group
    ensure_dir_exists(path)
    
    experiments = elab.get_experiments(tags=[tag_labcourseday, tag_group])
    experiments = [exp.id for exp in experiments]
    experiments.sort(reverse=True)
    
    with open(path+'/'+filename, 'w', encoding ='utf-8') as f:
        html_code = '<!DOCTYPE html>\n<html lang="de">\n<head>\n<meta charset="UTF-8">\n<script>\nMathJax = {\n  tex: {\n    inlineMath: [[\'$\', \'$\'], [\'\\\\(\', \'\\\\)\']],\n    displayMath: [[\'$$\',\'$$\'], [\'\\\\[\',\'\\\\]\']]\n  }\n};\n</script>\n<script id="MathJax-script" async\n  src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">\n</script>\n</head>\n<body>\n'
        html_code = html_code + f"<h1>Preparation of group {group}</h1>\n"
        html_code = html_code + f"<p>Excerpt from the electronic laboratory journal dated {(datetime.now().strftime('%d.%m.%Y, %H:%M.%S'))}</p>"
        for expid in experiments:
            exp = elab.get_experiment(expid)
            if exp.category_title == CATEGORY_EXPERIMENT:
                title = exp.title
                full_html = exp.body_html
                prep = full_html.split(PREP_UNTIL_HEADING)[0]
                prep = fixprep(prep, ignore=[IGNORE_MOTIVATION, IGNORE_PREPARATION, IGNORE_DESCRIPTION], shift_heading=2)
                prep = fiximages(prep, expid, path)
                prep = append_other_uploads(prep, expid, path)
                if prep != '':
                    html_code = html_code + '<h2>' + title + '</h2>' + prep
        html_code = html_code + '</body>\n</html>\n'
        html_code = html_code.replace('<hr>','')
        f.write(html_code)
        f.close()

    print(f'-- open in webbrowser: {(os.path.join(os.getcwd(), path, filename))}')
    print()
    webbrowser.open(os.path.join(os.getcwd(), path, filename))